# BinaryTwoStageDesigns - Quickstart

This is a [jupyter](http://jupyter.org) notebook using the [Julia](https://julialang.org/) kernel [IJulia.jl](https://github.com/JuliaLang/IJulia.jl) demonstrating the use of the Julia package [BinaryTwoStageDesigns](https://github.com/kkmann/BinaryTwoStageDesigns/blob/master/README.md).

In [1]:
using BinaryTwoStageDesigns, Cbc, Ipopt, DataFrames

## Setting

Assume that a new anti-cancer agent is to be tested against a historical response rate of $p_0=0.2$ in a phase-II trial and a response rate of $p_1=0.4$ is expected.
The maximal tolerable type-I-error rate for testing $\mathcal{H}_0:p\leq p_0$ is 5% and a type-II-error rate of 20% is deemed acceptable at $p_1=0.4$.

The corresponding single-stage design would require $n=47$ patients in this situation.

In [2]:
p0    = 0.2
p1    = 0.4
beta  = 0.2
alpha = 0.05

0.05

## Adaptive Design

Alternatively, a two-stage adaptive design could be used which minimizes the expected sample size under $p_1=0.4$ subject to the same constraints. 
Additionally, for operational reasons a potential second stage must enroll at least 5 patients. Also, upon rejection of the null hypothesis, at least 25 patients must be enrolled to ensure a sufficiently precise effect estimate for subsequent phase-III planning.

### Sample Space

First, a sample space object is defined. It simply holds infomarion about the allowable search space for the optimization algorithm. Here, the range of possible stage-one sample sizes is limited to 10 to 20 and the maximal overall sample size to 75. 

In [3]:
s = SampleSpace(
    15:25, # n1 range
    75     # nmax
)

### Parameters

Next, the design parameters are also stored in an object. The simplest parameters object corresponds to minimising expected sample size. For a `MESS`-object only $p_0, p_1$, maximal type one and two error rates and the parameter value for which the expected sample size is to be minimized are required besides the sample space object created earlier.

In [4]:
params1 = MESS(
    s,                          # sample space
    p0, p1;                     # null and planning alternative
    alpha = alpha, beta = beta, # max. type one and two error rates
    pess = p1                   # alternative on which to minimize expected sample size
)

SampleSpace

### Optimization

Finally, a solver can be defined and the optimization process is started. Note that both the optimal design as well as all design found while exhaustively exploring the $n_1$-space are returned. The basic technique via integer programming has been desribed in [Kunzmann & Kieser 2016](https://arxiv.org/abs/1605.00249).

In [5]:
design1, res1 = optimaldesign(params1, CbcSolver(seconds = 300), VERBOSE = 1)
DataFrame(design1)

MESS
optimizing design for parameters ''
considering 11 stage-one sample sizes between 15 and 25 using Cbc.CbcMathProgSolverInterface.CbcSolver as solver

    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
17:52:10    15      9.1             164               2.7   +2.45e+01   +2.45e+01              0.0


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
17:54:54    16     18.2              61               3.7   +2.43e+01   +2.43e+01              0.0


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
17:55:54    17     27.3              99               5.4   +2.47e+01   +2.43e+01              1.5


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
17:57:34    18     36.4              53               6.3   +2.46e+01   +2.43e+01              0.9


    time    n1   % done   sol. time [s]   cum


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:02:43    22     72.7             302              15.6   +2.92e+01   +2.43e+01             19.9


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:07:45    23     81.8             131              17.8   +2.57e+01   +2.43e+01              5.7


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:09:57    24     90.9              73              19.0   +2.63e+01   +2.43e+01              8.0


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:11:10    25    100.0             110              20.8   +2.70e+01   +2.43e+01             11.0



done after 21 minutes.
Optimal stage-one sample size is 16 resulting in a minimal score of 2.43e+01



,x1,n,c
1,0,16,Inf
2,1,16,Inf
3,2,16,Inf
4,3,16,Inf
5,4,44,14.0
6,5,34,11.0
7,6,29,9.0
8,7,16,-Inf
9,8,16,-Inf
10,9,16,-Inf


#### Bayesian approach

Alternatively, a Bayesian design criterion can be used where the expected sample size under a prior distribution is minimized subject to a constraint on expected power.
To this end, the minimal clinically relevant response rate $p_{MCR}$ must be defined. 
Here we assume that $p_{MCR}=p_0+0.1$.

In [6]:
pmcr = p0 + .1

0.30000000000000004

For the prior, we simply define a Beta distribution with mass centered slightly below $0.4$:

In [7]:
f(p) = Distributions.pdf(Distributions.Beta(5, 8), p)

f (generic function with 1 method)

Also, for operational reasons additional constraints on the feasible region, i.e., the sample space are imposed.
Often, it will be sensible to require a certain minimal number of subjects for the second stage to outweight the operational burden of an interim analysis (here: 5) and to require a certain minimal number upon rejection of the null hypothesis to ensure a sufficient precision of the response rate estimate when going on to a subsequent phase III trial (here: 25).

In [8]:
s2 = SampleSpace(
    15:25,
    75,   
    n2min = 5, # minmal stage-two sample size
    nmincont = 25 # minimal sample size upon rejection of the null
)

params2 = MBESS(
    s2,                       # sample space
    p0, pmcr, f,          # null, pmcrv, and prior
    alpha = alpha, beta = beta # max. type one error rate and expected type two error rates
)

In [9]:
design2, res2 = optimaldesign(params2, CbcSolver(seconds = 300), VERBOSE = 1)
DataFrame(design2)

MBESS
optimizing design for parameters ''
considering 11 stage-one sample sizes between 15 and 25 using Cbc.CbcMathProgSolverInterface.CbcSolver as solver

    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:13:07    15      9.1              31               0.5   +2.56e+01   +2.56e+01              0.0




    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:13:38    16     18.2             307               5.6   +2.61e+01   +2.56e+01              2.1


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:18:46    17     27.3              75               6.9   +2.55e+01   +2.55e+01              0.0


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:20:01    18     36.4              48               7.7   +2.55e+01   +2.55e+01              0.0


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:20:49    19     45.5              88               9.2   +2.57e+01   +2.55e+01              0.8


    time    n1   % done   sol. time [s]   cum. time [min]       score        best   % diff to best
18:22:17    20     54.5              58              10.1   +2.55e+01   +2.55e+01              0.1



,x1,n,c
1,0,18,Inf
2,1,18,Inf
3,2,18,Inf
4,3,18,Inf
5,4,18,Inf
6,5,35,11.0
7,6,32,10.0
8,7,26,8.0
9,8,25,-Inf
10,9,25,-Inf


## Inference

After completing the trial with 5/16 responses in stage one and 10 further responses in stage two, a point estimate and confidence iterval are required. Point estimates were discussed in [Kunzmann & Kieser 2016](http://onlinelibrary.wiley.com/doi/10.1002/sim.7200/abstract) and different estimators are implemented. Here, we use a compatible minimum expected mean squared error estimator with several favorable properties.

In [10]:
est = OCEstimator(design1, IpoptSolver())

estimate(est, 5, 10)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:      154
Number of nonzeros in Lagrangian Hessian.............:    11250

Total number of variables............................:       78
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       78
                     variables with only upper bounds:        0
Total number of equa

0.43695005144224414

The maximum likelihood estimator (MLE) would have been:

In [11]:
(5 + 10) / (samplesize(design1, 5))

0.4411764705882353

Any estimator induces an ordering on the sample space which in turn implies p values. The major advantage of the novel optimal compatible estimators in [Kunzmann & Kieser 2016](http://onlinelibrary.wiley.com/doi/10.1002/sim.7200/abstract) is the fact that their implied p values are *always* compatible with the design's test decision.

In [12]:
pvalue(est, 5, 10, p0)

0.00750164919381819

The very same ordering/p values can then be used to derive a Clopper-Pearson type confidence interval (paper under review):

In [13]:
ci = ECPInterval(est, confidence = .9)

limits(ci, 5, 10)

2-element Array{Float64,1}:
 0.271
 0.647